In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

japanize_matplotlib is already installed.
catboost is already installed.
optuna is already installed.
geolib is already installed.
Directory and file for OpenCL vendors have been created.


In [ ]:
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

In [ ]:
from utils import mae_loss_fn, WeightsSearcher,to_cat_col

In [ ]:
#TARGET="単位面積あたりの取引価格_log"
TARGET="target"

#oof_true_df=df_train[[f"{TARGET}"]]

oof_lgbm_df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/oof_lgbm_df.csv")
oof_lgbm_df=oof_lgbm_df.set_index("ID")

#oof_cat_df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/oof_cat_df.csv")
oof_cat_df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/v1_oof_cat_df.csv")
oof_cat_df=oof_cat_df.set_index("ID")

#print("MAE_lgbm",mean_absolute_error(oof_true_df[TARGET],oof_lgbm_df["pred_lgbm"]))
#print("MAE_cat",mean_absolute_error(oof_true_df[TARGET],oof_cat_df["pred_cat"]))

oof_df=pd.concat([oof_lgbm_df,oof_cat_df],axis=1)

# oof_df から重複した "target" 列を削除
oof_df = oof_df.loc[:,~oof_df.columns.duplicated()]

print("MAE_lgbm",mean_absolute_error(oof_df[TARGET],oof_df["pred_lgbm"]))
print("MAE_cat",mean_absolute_error(oof_df[TARGET],oof_df["pred_cat"]))

TARGET="target"

print(oof_lgbm_df)
print(oof_cat_df)
print(oof_df)

MAE_lgbm 0.06585289825284472
MAE_cat 0.06609169952321634
            target  pred_lgbm
ID                           
1046933   5.243038   5.329627
1079053   5.124939   5.059953
1170588   4.867762   5.075637
1001637   5.484674   5.486013
1014790   5.260601   5.267016
...            ...        ...
47001427  5.342423   5.369479
47035975  5.726999   5.687249
47003811  5.528274   5.657619
47010902  5.522879   5.534073
47011639  5.329059   5.422368

[1009271 rows x 2 columns]
            target  pred_cat
ID                          
1046933   5.243038  5.315122
1079053   5.124939  5.066515
1170588   4.867762  5.096096
1001637   5.484674  5.507657
1014790   5.260601  5.260658
...            ...       ...
47001427  5.342423  5.378117
47035975  5.726999  5.714843
47003811  5.528274  5.620197
47010902  5.522879  5.511248
47011639  5.329059  5.420627

[1009271 rows x 2 columns]
            target  pred_lgbm  pred_cat
ID                                     
1046933   5.243038   5.329627  5.315122


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
#df=df.drop(columns=["住所_地区名欠損フラグ","address","title",PRICE_PER_AREA_LOG],axis=1)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log',"単位面積あたりの取引価格_log", '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]

# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

#print("Train DataFrame:")
#print(df_train)
#print("\nTest DataFrame:")
#print(df_test)

oof_lgbm_target_df=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/oof_lgbm_target_df.csv")
oof_lgbm_target_df=oof_lgbm_target_df.set_index("ID")
oof_lgbm_target_df=oof_lgbm_target_df.reindex(oof_lgbm_df.index)

df_train_target=df_train.copy()

df_train_target=df_train_target.reindex(oof_lgbm_target_df.index)
#print(df_train_target)

df_train_target=df_train_target.copy()
#df_train_target=df_train_target.rename(columns={"target":"target_tg"})

df_train_target_=pd.concat([df_train_target,oof_lgbm_target_df],axis=1)
df_train_target_=df_train_target_.rename(columns={"target":"target_tg"})
df_train_target_=df_train_target_.rename(columns={"pred_lgbm":"pred_tg_lgbm"})

df_train_target_=pd.concat([df_train_target_,oof_lgbm_df],axis=1)
df_train_target_=pd.concat([df_train_target_,oof_cat_df],axis=1)

#df_train_target_["pred_tg_lgbm"]=np.log10(10**df_train_target_["pred_tg_lgbm"]/df_train_target_["面積（㎡）"])#####
#df_all.rename(columns={"取引価格（総額）_log":"単位面積あたりの取引価格_log"},inplace=True)

# 単位面積あたりの取引価格の対数から10のべき乗を取る
#price_per_area = 10**y_pred_lgbm
# その値に面積を掛けて、取引価格（総額）を計算
#predicted_price_total = price_per_area * df_test_tree00["面積（㎡）"]
# 取引価格（総額）の対数を計算
#y_pred= np.log10(predicted_price_total)

df_train_target_["pred_lgbm"]=np.log10(10**df_train_target_["pred_lgbm"]*df_train_target_["面積（㎡）"])
df_train_target_["pred_cat"]=np.log10(10**df_train_target_["pred_cat"]*df_train_target_["面積（㎡）"])


print("MAE_lgbm",mean_absolute_error(df_train_target_["target_tg"],df_train_target_["pred_lgbm"]))
print("MAE_tg_lgbm",mean_absolute_error(df_train_target_["target_tg"],df_train_target_["pred_tg_lgbm"]))
print("MAE_cat",mean_absolute_error(df_train_target_["target_tg"],df_train_target_["pred_cat"]))

print(df_train_target_)

MAE_lgbm 0.06585289825284472
MAE_tg_lgbm 0.06636916319379245
MAE_cat 0.06609169952321634
             価格情報区分 都道府県名   市区町村名    地区名  住所不一致_Flag              住所 最寄駅：名称  \
ID                                                                            
1046933   不動産取引価格情報   北海道  札幌市白石区    南郷通           0    北海道札幌市白石区南郷通  南郷７丁目   
1079053   不動産取引価格情報   北海道   札幌市西区  発寒１２条           0   北海道札幌市西区発寒１２条   発寒中央   
1170588      成約価格情報   北海道     江別市   大麻北町           0      北海道江別市大麻北町   森林公園   
1001637      成約価格情報   北海道  札幌市中央区   北５条西           0   北海道札幌市中央区北５条西  西２８丁目   
1014790   不動産取引価格情報   北海道  札幌市中央区  宮の森２条           0  北海道札幌市中央区宮の森２条  西２８丁目   
...             ...   ...     ...    ...         ...             ...    ...   
47001427  不動産取引価格情報   沖縄県     那覇市  首里山川町           0     沖縄県那覇市首里山川町     儀保   
47035975     成約価格情報   沖縄県  中頭郡北谷町     美浜           0     沖縄県中頭郡北谷町美浜  てだこ浦西   
47003811  不動産取引価格情報   沖縄県     那覇市    古波蔵           0       沖縄県那覇市古波蔵     壺川   
47010902  不動産取引価格情報   沖縄県     浦添市     宮城  

In [ ]:
df_train_target_=df_train_target_.drop("target",axis=1)
df_train_target_=df_train_target_.rename(columns={"target_tg":"target"})

oof_df=df_train_target_[['target','pred_cat', 'pred_lgbm','pred_tg_lgbm']]
oof_df

,target,pred_cat,pred_lgbm,pred_tg_lgbm
ID,,,,
1046933,7.146128,7.218212,7.232717,7.214925
1079053,7.146128,7.087704,7.081142,7.156613
1170588,6.770852,6.999186,6.978727,7.000789
1001637,7.462398,7.485380,7.463736,7.473024
1014790,6.913814,6.913871,6.920229,6.866648
...,...,...,...,...
47001427,7.342423,7.378117,7.369479,7.380612
47035975,7.602060,7.589904,7.562310,7.556450
47003811,7.431364,7.523287,7.560709,7.559929


In [ ]:
def mae_loss_fn(true_targets,pred_weighted):
    return mean_absolute_error(true_targets,pred_weighted)

class WeightsSearcher:
    def __init__(self,loss_fn,bounds=[],mode="min",method="SLSQP"):
        self.loss_fn=loss_fn#損失関数
        self.bounds=bounds#重みの範囲
        self.mode=mode#最適化モード(minまたはmax)
        self.method=method#最適化方法(デフォルトは"SLSQP")

    # 目的関数のラッパー: 予測値、真のターゲット、目的関数を受け取り、重み付けされた予測値を計算する関数を返す
    def _objective_function_wrapper(self,pred_values,true_targets,obj_fn):
        def objective_function(weights):
            #print(pred_values)
            #予測値に重みを適用し、行ごとに合計
            pred_weighted=(pred_values*weights).sum(axis=1)
            #真のターゲットと重み付けされた予測値を使用してスコアを計算
            score = obj_fn(true_targets, pred_weighted)
            # 最適化モードに応じてスコアを反転
            return score if self.mode == "min" else score
        #目的関数を返す
        return objective_function

    #最適な重みを見つけるメソッド:検証予測値と真のターゲットを受け取り、最適化を実行
    def find_weights(self, val_preds, true_targets):
        len_models = len(self.bounds)#モデルの数を取得
        bounds = [0,1] * len_models if len(self.bounds) == 0 else self.bounds# 重みの範囲が指定されていない場合は [0,1] の範囲を設定
        ##np.ones は、指定された形状とデータ型で、すべての要素が1である新しい配列を生成するNumPy関数です。
        ##len_models はモデルの数を表しており、この数だけ1が含まれた配列が作成されます。
        ##例えば、len_models が3の場合、np.ones(len_models) は [1, 1, 1] という配列を生成します。
        ###initial_weights = np.ones(len_models) / len_models# 初期重みを設定（各モデルに等しい重み）
        initial_weights = np.random.rand(len_models)  # 初期重みをランダムに設定
        initial_weights /= np.sum(initial_weights)  # 初期重みを正規化
        objective_function = self._objective_function_wrapper(val_preds, true_targets, self.loss_fn)#目的変数をラップ

        # 制約条件: 重みの合計が1になるようにする
        constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1})

        result = minimize(
            objective_function,
            initial_weights,
            bounds=bounds,
            method=self.method,
            constraints=constraints
        )
        #最適化された重みを取得
        optimized_weights = result.x
        #重みの合計が1になるように正規化
        optimized_weights /= np.sum(optimized_weights)
        #最適化された重みを返す
        return optimized_weights

#pred_cols=["pred_cat","pred_lgbm"]
pred_cols=["pred_cat","pred_lgbm","pred_tg_lgbm"]
#pred_cols=["pred_cat","pred_lgbm","pred_xg"]
#model_names=["cat","lgbm"]
model_names=["cat","lgbm","tg_lgbm"]
#model_names=["cat","lgbm","xg"]

#それぞれのモデルの予測値の範囲を設定
bounds=[(0,1)]*len(pred_cols)
print(bounds)
# ROC AUCスコアの補数を計算する関数
mae_loss_fn=lambda y_true, y_pred:mean_absolute_error(y_true,y_pred)

w_searcher = WeightsSearcher(mae_loss_fn, bounds
                             ,method='Nelder-Mead'
                             #,method="L-BFGS-B"
                             #,method='COBYLA'
                             ) # log_loss, gini_stability roc_auc_fn
optimized_weights = w_searcher.find_weights(
    oof_df[pred_cols].to_numpy(),
    oof_df[f"{TARGET}"]
)

optimized_weights_df = pd.DataFrame(zip(model_names, optimized_weights), columns=['model', 'weight'])
display(optimized_weights_df)
print("sum: ", np.sum(optimized_weights))

#-----------------
train_preds_df=oof_df.copy()
train_preds_df["pred_weight"]=(optimized_weights*train_preds_df[pred_cols]).sum(axis=1).to_numpy()
#display(train_preds_df)

# 重みを取得
weights = optimized_weights_df.set_index('model')['weight'].to_dict()

mae=mean_absolute_error(train_preds_df["target"],train_preds_df["pred_weight"])
print("MAE:",mae)
print("cat_weight:",weights["cat"])
print("lgbm_weight:",weights["lgbm"])
print("lgbm_tg_weight:",weights["tg_lgbm"])

[(0, 1), (0, 1), (0, 1)]


<ipython-input-15-1524cfb640f5>:39: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  result = minimize(


,model,weight
0,cat,0.414725
1,lgbm,0.332699
2,tg_lgbm,0.252575


sum:  1.0
MAE: 0.06483274035179776
cat_weight: 0.41472518215158677
lgbm_weight: 0.3326993991568465
lgbm_tg_weight: 0.2525754186915667


In [ ]:
df_arima=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/df_arima_mean.csv")
print(df_arima)

# 71番目の値を72番目の値で割る
time_weight = 1+(1-df_arima.loc[71, '単位面積あたりの取引価格_log_mean'] / df_arima.loc[72, '単位面積あたりの取引価格_log_mean'])
print(time_weight)

       取引時点  単位面積あたりの取引価格_log_mean
0   2005.75               5.603129
1   2005.99               5.600274
2   2006.25               5.610975
3   2006.50               5.505956
4   2006.75               5.511552
..      ...                    ...
69  2022.99               5.617195
70  2023.25               5.624689
71  2023.50               5.620718
72  2023.75               5.629596
73  2023.99               5.634549

[74 rows x 2 columns]
1.0015771930023731


In [ ]:
print("MAE:",mae)
print("cat_weight:",weights["cat"])
print("lgbm_weight:",weights["lgbm"])
print("lgbm_tg_weight:",weights["tg_lgbm"])

df_test_lgbm=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_lgbm_cv20240908_111634.csv")
df_test_lgbm=df_test_lgbm.set_index("ID")

#df_test_cat=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_cat_cv20240908_141325修正.csv")
df_test_cat=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/final_predictions/v1test_cat_cv20240919_111048.csv")
df_test_cat=df_test_cat.set_index("ID")

df_test_lgbm_tg=pd.read_csv(r"/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_l1_tg20240910_144521.csv")
df_test_lgbm_tg=df_test_lgbm_tg.set_index("ID")

#df_pred_test=df_test_cat*weights["cat"]+df_test_lgbm*weights["lgbm"]
df_pred_test=df_test_cat*weights["cat"]+df_test_lgbm*weights["lgbm"]+df_test_lgbm_tg*weights["tg_lgbm"]
df_pred_test=df_pred_test*time_weight
df_pred_test.to_csv(r"/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_lgbm_lgbm_tg_cat_NMtime.csv")

df_pred_test

MAE: 0.06483274035179776
cat_weight: 0.41472518215158677
lgbm_weight: 0.3326993991568465
lgbm_tg_weight: 0.2525754186915667


,取引価格（総額）_log
ID,
1000107,6.737715
1000109,7.319648
1000112,6.638234
1000114,6.468357
1000117,7.407072
...,...
47036620,7.544982
47037115,7.484123
47037850,7.287747
